In [1]:
import urllib
import pandas as pd 
from sunpy.time import TimeRange
from flarelist_utils import read_swpc_reports, read_ngdc_goes_reports
from dateutil.relativedelta import relativedelta
import pandas as pd 
import numpy as np
import datetime
import glob
import h5netcdf
import matplotlib.pyplot as plt
from scipy.io import readsav
import warnings
warnings.filterwarnings("ignore")

In [2]:
def print_flares(df):
    x = np.sum(df["goes_class_ind"].isin(["X"]))
    m = np.sum(df["goes_class_ind"].isin(["M"]))
    c = np.sum(df["goes_class_ind"].isin(["C"]))
    print("X: {:d}, M: {:d}, C: {:d}".format(x, m, c))

In [3]:
def get_yearly_swpc(year):
    year = datetime.datetime(year, 1, 1)
    filedir = "./goes_files/%Y_events/%Y*events*"
    all_files = []
    all_files += glob.glob(year.strftime(filedir))
    all_files.sort()
    df_flares = read_swpc_reports(all_files[0])
    for f in all_files[1:]:
        df = read_swpc_reports(f)
        df_flares = df_flares.append(df)
    df_flares.reset_index(inplace=True, drop=True)
    df_flares["ts"] = df_flares.date + df_flares.start_time
    df_flares.drop_duplicates(subset="ts")
    return df_flares[df_flares["goes_class_ind"].isin(["C", "X", "M"])]


def get_yearly_ngdc(year):
    year = datetime.datetime(year, 1, 1)
    file = "./goes_files/goes-xrs-report_%Y.txt"
    df = read_ngdc_goes_reports(year.strftime(file))
    df["ts"] = df.date + df.start_time
    df.drop_duplicates(subset="ts")
    return df[df["goes_class_ind"].isin(["X", "M", "C"])]

def get_yearly_ssw(year):
    tstart = datetime.datetime(year, 1, 1)
    tend = datetime.datetime(year, 12, 31)
    return ssw[(ssw["datetime"]>=tstart) & (ssw["datetime"]<=tend)]
    
def get_yearly_hek(year):
    tstart = datetime.datetime(year, 1, 1)
    tend = datetime.datetime(year, 12, 31)
    return hek_flares[(hek_flares["datetime"]>=tstart) & (hek_flares["datetime"]<=tend)]

In [4]:
ssw_flarelist = pd.read_csv("full_sswlatest.csv")
ssw = ssw_flarelist[ssw_flarelist["goes_class_ind"].isin(["X", "M", "C"])]
ssw["datetime"] = pd.to_datetime(ssw["time_start"])

In [5]:
len(ssw)

8773

In [6]:
print(ssw.datetime.min(), ssw.datetime.max())

2010-01-02 07:09:00 2018-03-30 07:57:00


## Lets look at HEK flares

In [7]:
hek_flares_all = pd.read_csv("solar_cycle24_flares.csv")
hek_flares_all.head(2)

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum
0,2009-07-05T07:07:00,2009-07-05T07:13:00,2009-07-05T07:18:00,C2.7,11024
1,2009-07-06T16:59:00,2009-07-06T17:05:00,2009-07-06T17:11:00,C1.0,11024


In [8]:
hek_flares_all["goes_class_ind"] = [x[0] for x in hek_flares_all["fl_goescls"]]
hek_flares_all["datetime"] = pd.to_datetime(hek_flares_all["event_starttime"])
hek_flares = hek_flares_all[hek_flares_all["goes_class_ind"].isin(["X", "M", "C"])]

In [9]:
hek_flares = hek_flares[(hek_flares.datetime>="2010-01-01")&(hek_flares.datetime<="2018-12-31")]

In [10]:
hek_flares.reset_index(drop=True, inplace=True)

In [11]:
hek_flares = hek_flares.drop_duplicates(subset="event_starttime")
len(hek_flares)

8619

In [12]:
len(hek_flares.drop_duplicates(subset="event_starttime"))

8619

In [13]:
print("SSW from helio:"); print_flares(ssw)
print("HEK:"); print_flares(hek_flares)

SSW from helio:
X: 49, M: 735, C: 7989
HEK:
X: 49, M: 756, C: 7814


In [14]:
def check_for_years(year):
    print(year)
    print_flares(get_yearly_hek(year))
    print_flares(get_yearly_ssw(year))
    print_flares(get_yearly_ngdc(year))
    print_flares(get_yearly_swpc(year))

In [15]:
for i in [2010, 2011, 2012, 2013, 2014, 2015, 2016]:
    check_for_years(i)

2010
X: 0, M: 23, C: 169
X: 0, M: 19, C: 161
X: 0, M: 23, C: 170
X: 0, M: 23, C: 170
2011
X: 8, M: 109, C: 1199
X: 8, M: 105, C: 1254
X: 8, M: 111, C: 1200
X: 8, M: 111, C: 1200
2012
X: 7, M: 129, C: 1336
X: 7, M: 112, C: 1216
X: 7, M: 123, C: 1259
X: 7, M: 129, C: 1337
2013
X: 12, M: 98, C: 1349
X: 12, M: 96, C: 1417
X: 12, M: 99, C: 1356
X: 12, M: 98, C: 1353
2014
X: 16, M: 209, C: 1785
X: 16, M: 210, C: 1906
X: 16, M: 205, C: 1797
X: 16, M: 207, C: 1798
2015
X: 2, M: 130, C: 1378
X: 2, M: 132, C: 1411
X: 2, M: 104, C: 1301
X: 2, M: 125, C: 1377
2016
X: 0, M: 16, C: 324
X: 0, M: 17, C: 343
X: 0, M: 16, C: 302
X: 0, M: 16, C: 321


## Lets look at the flares from the GOES IDL workbench

```
a = ogoes()
gev = a->get_gev('01-Jan-2010', '31-Dec-2018', /struct)
save, gev, filename="goes_flares_from_idl.sav"
```

In [16]:
gev_data = readsav("goes_flares_from_idl.sav")["gev"]

In [17]:
gev_dict = {k : gev_data[k].astype(str) for k in gev_data.dtype.names}
gev_df = pd.DataFrame(gev_dict)

In [18]:
gev_df["goes_class_ind"] = [x[0] for x in gev_df["CLASS"]]
gev_df["datetime"] = pd.to_datetime(gev_df["GSTART"])

In [19]:
gev_df_c = gev_df[gev_df["goes_class_ind"].isin(["C", "M", "X"])]
print(gev_df_c.datetime.min(), gev_df_c.datetime.max())

2010-01-02 07:09:00 2018-07-06 19:41:00


In [20]:
print("GEV sswidl:"); print_flares(gev_df_c)
print("SSW from helio:"); print_flares(ssw)
print("HEK:"); print_flares(hek_flares)

GEV sswidl:
X: 49, M: 740, C: 7736
SSW from helio:
X: 49, M: 735, C: 7989
HEK:
X: 49, M: 756, C: 7814


In [21]:
def ryan_test():
    gev_df = gev_df_c[(gev_df_c.datetime>"2010-05-01")&(gev_df_c.datetime<"2016-10-31")]
    sswy = ssw[(ssw.datetime>"2010-05-01")&(ssw.datetime<"2016-10-31")]
    print_flares(gev_df)
    print_flares(sswy)
ryan_test()

X: 45, M: 686, C: 7390
X: 45, M: 679, C: 7639


In [ ]:
year = 2016
ssw_test = get_yearly_ssw(year)
ngdc_test = get_yearly_ngdc(year)
swpc_test = get_yearly_swpc(year)

In [ ]:
ssw_test_m = ssw_test[ssw_test["goes_class_ind"].isin(["M"])]; ssw_test_m.reset_index(inplace=True, drop=True)
ngdc_test_m = ngdc_test[ngdc_test["goes_class_ind"].isin(["M"])]
swpc_test_m = swpc_test[swpc_test["goes_class_ind"].isin(["M"])]

In [ ]:
ssw_test_m.iloc[8:12]

In [ ]:
swpc_test_m.iloc[7:11]

## Test new "reprocessed data"
https://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes14/xrsf-l2-flsum_science/
https://satdat.ngdc.noaa.gov/sem/goes/data/science/xrs/goes15/xrsf-l2-flsum_science/

Looks like these are the rescaled data so not what we need here

In [ ]:
test = h5netcdf.File("sci_xrsf-l2-flsum_g15_s20100331_e20200304_v1-0-0.nc")

In [ ]:
list(test.variables)

In [ ]:
np.array(test.variables["flare_counter"])

In [ ]:
flare_class = np.array(test["flare_class"]).astype(str)

In [ ]:
set(flare_class)

In [ ]:
np.array(test["status"])

In [ ]:
plt.plot(np.array(test["time"]))

## HEK testing sunpy

In [ ]:
from sunpy.net import Fido, attrs as a


In [ ]:
res = Fido.search(a.Time("2010-01-01", "2018-12-31"), 
                  a.hek.EventType("FL"), a.hek.FRM.Name=="SSW Latest Events")

In [ ]:
res2 = res[0].to_pandas()

In [ ]:
res2["frm_name"].unique()

In [ ]:
res2["fl_goescls"].unique()

In [ ]:
res2["obs_instrument"].unique()

In [ ]:
list(res2["event_starttime"].unique())

In [ ]:
"SSW Latest Events"  in res

In [ ]:
res["f"

In [ ]:
def get_flares():
    """
    Query HEK for flares > C1 from past solar cycle and save results to csv.
    
    """
    event_type = "FL"
    tstart = "2010/01/01"
    tend = "2018/12/31"
    result = Fido.search(a.Time(tstart, tend),
                         a.hek.EventType(event_type),
                         a.hek.FRM.Name == "SWPC", 
                         a.hek.FL.GOESCls >= "C1.0")
    
    new_table = result["hek"]["event_starttime", "event_peaktime",
                             "event_endtime", "fl_goescls", "ar_noaanum", "frm_name",
                              "obs_observatory", "frm_institute", "search_frm_name"]
    new_table.write("solar_cycle24_flares.csv", format="csv")

In [ ]:
event_type = "FL"
tstart = "2010/01/01"
tend = "2018/12/31"
result = Fido.search(a.Time(tstart, tend),
                     a.hek.EventType(event_type),
                     a.hek.FRM.Name == "SWPC", 
                     a.hek.FL.GOESCls >= "C1.0")

new_table = result["hek"]["event_starttime", "event_peaktime",
                         "event_endtime", "fl_goescls", "ar_noaanum", "frm_name",
                          "obs_observatory", "frm_institute", "search_frm_name"]
new_table.write("solar_cycle24_flares_swpc.csv", format="csv")

In [ ]:
ls
